In [2]:
import numpy as np
import cv2
from sklearn.cluster import KMeans
from collections import Counter
import imutils
import pprint
from matplotlib import pyplot as plt

In [3]:
def removeBlack(estimator_labels, estimator_cluster):

    hasBlack = False
    occurance_counter = Counter(estimator_labels)
    def compare(x, y): return Counter(x) == Counter(y)
    for x in occurance_counter.most_common(len(estimator_cluster)):

        color = [int(i) for i in estimator_cluster[x[0]].tolist()]
        if compare(color, [0, 0, 0]) == True:
            del occurance_counter[x[0]]
            hasBlack = True
            estimator_cluster = np.delete(estimator_cluster, x[0], 0)
            break

    return (occurance_counter, estimator_cluster, hasBlack)

In [4]:
def getColorInformation(estimator_labels, estimator_cluster, hasThresholding=False):

    occurance_counter = None
    colorInformation = []
    hasBlack = False
    if hasThresholding == True:

        (occurance, cluster, black) = removeBlack(
            estimator_labels, estimator_cluster)
        occurance_counter = occurance
        estimator_cluster = cluster
        hasBlack = black

    else:
        occurance_counter = Counter(estimator_labels)

    totalOccurance = sum(occurance_counter.values())
    for x in occurance_counter.most_common(len(estimator_cluster)):

        index = (int(x[0]))
        index = (index-1) if ((hasThresholding & hasBlack)
                              & (int(index) != 0)) else index
        color = estimator_cluster[index].tolist()
        color_percentage = (x[1]/totalOccurance)
        colorInfo = {"cluster_index": index, "color": color,
                     "color_percentage": color_percentage}
        colorInformation.append(colorInfo)

    return colorInformation



In [5]:
def extractDominantColor(image, number_of_colors=5, hasThresholding=False):

    if hasThresholding == True:
        number_of_colors += 1
    img = image.copy()
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.reshape((img.shape[0]*img.shape[1]), 3)
    estimator = KMeans(n_clusters=number_of_colors, random_state=0)
    estimator.fit(img)
    colorInformation = getColorInformation(
        estimator.labels_, estimator.cluster_centers_, hasThresholding)
    return colorInformation

In [6]:
def plotColorBar(colorInformation):
    color_bar = np.zeros((100, 500, 3), dtype="uint8")
    top_x = 0
    for x in colorInformation:
        bottom_x = top_x + (x["color_percentage"] * color_bar.shape[1])
        color = tuple(map(int, (x['color'])))
        cv2.rectangle(color_bar, (int(top_x), 0),
                      (int(bottom_x), color_bar.shape[0]), color, -1)
        top_x = bottom_x
    return color_bar

In [7]:
image = cv2.imread('./dataset/img1.jpg')

In [8]:
image = imutils.resize(image,width=500)

In [9]:
cv2.imshow('Original',image)
cv2.waitKey()
cv2.destroyAllWindows()

In [10]:
img = image.copy()
img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
lower_threshold = np.array([0, 48, 80], dtype=np.uint8)
upper_threshold = np.array([20, 255, 255], dtype=np.uint8)
skinMask = cv2.inRange(img, lower_threshold, upper_threshold)
skinMask = cv2.GaussianBlur(skinMask, (3, 3), 0)
skin = cv2.bitwise_and(img, img, mask=skinMask)
skin = cv2.cvtColor(skin, cv2.COLOR_HSV2BGR)

In [11]:
cv2.imshow('Thresholded Image',skin)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
dominantColors = extractDominantColor(skin, hasThresholding=True)

In [ ]:
print("Color Information")
for x in dominantColors:
        print(pprint.pformat(x))
        print()

In [ ]:
colour_bar = plotColorBar(dominantColors)
plt.subplot(3, 1, 3)
plt.axis("off")
plt.imshow(colour_bar)
plt.title("Color Bar")

In [ ]:
plt.tight_layout()
plt.show()